# DataFrames Basics

## Prerrequisites

Install Spark and Java in VM

In [2]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [3]:
ls -l # check the .tgz is there

total 399736
-rw-r--r-- 1 root root   8926788 Feb  5 19:49 fifa_players.csv
drwxr-xr-x 1 root root      4096 Feb  2 14:53 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz


In [4]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [5]:
!pip install -q findspark

In [6]:

!pip install py4j

# For maps
!pip install folium
!pip install plotly

Define the environment

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [8]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [9]:
spark

In [10]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [11]:
# Import sql functions
from pyspark.sql.functions import *

# **HIPÓTESIS: Los mejores jugadores juegan en la Premier League.**



A continuación, se va a realizar un análisis a través de spark sql para demostrar que los mejores jugadores están jugando en la liga inglesa, que no jugadores ingleses. Para ello, se tendrán en cuenta los parámetros de la valoración (overall) y la liga (obtenida a través de los clubes).

In [12]:
import matplotlib.pyplot as plt

In [13]:
futbolDF = spark.read.option("header", "true").csv("/fifa_players.csv")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/fifa_players.csv.

In [ ]:
futbolDF.show()

A continuación, comenzamos con un análisis de los mejores 25 jugadores. Para ello requerimos un desglose que nos muestre la valoración del jugador acompañado de a que club pertenece.

In [ ]:
futbolDF.select("short_name","overall", "club").show(25)

En el próximo paso, veremos que equipos poseen algún jugador dentro de este top 25 de una manera más visual y sencilla.

In [ ]:
top_25_teams = futbolDF.limit(25).groupBy("club").count()


In [ ]:
top_25_teams_pd = top_25_teams.toPandas()

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(top_25_teams_pd["club"], top_25_teams_pd["count"], color='green')
plt.xlabel("Equipo")
plt.ylabel("Número de Jugadores en Top 25")
plt.title("Número de Jugadores por Equipo en el Top 25")
plt.xticks(rotation=45, ha="right")
plt.show()

Aqui vemos como esta la distribución de esos 25 jugadores donde vemos que los equipos ingleses son, excepto el Liverpool que posee 3 jugadores el resto esta por debajo de otros como equipos españoles e italianos.

Ahora queremos calcular la media de ese overall de todos los equipos en función de los jugadores que tengan en el top 25

In [ ]:
top_25_teams_overall = futbolDF.limit(25).groupBy("club").agg(avg(col("overall")).alias("media_overall"))

In [ ]:
top_25_teams_overall.show()

Queremos saber si esa relación de puntos de equipo esta relacionado con el valor de esos jugadores y el salario que reciban.

In [ ]:
result_df = futbolDF.select("short_name", "overall", "club", "value_eur", "wage_eur")

In [ ]:
result_df.show(25, truncate=False)

Agrupamos para ver si los mejores clubes son los que mejor pagan y mas salario en total poseen

In [ ]:
top_25_players_overall = futbolDF.orderBy(col("overall").desc()).limit(25)

In [ ]:
suma_por_equipo_top_25 = top_25_players_overall.select("club", "value_eur", "wage_eur") \
    .groupBy("club") \
    .agg(sum(col("value_eur")).alias("suma_valor_equipo"), sum(col("wage_eur")).alias("suma_salarios_equipo"))

In [ ]:
suma_por_equipo_top_25.show(truncate=False)

Vamos a graficar esta información

In [ ]:
pandas_df = suma_por_equipo_top_25.toPandas()

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Equipo')
ax1.set_ylabel('Suma Valor Equipo', color=color)
ax1.bar(pandas_df['club'], pandas_df['suma_valor_equipo'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Suma Salarios Equipo', color=color)
ax2.plot(pandas_df['club'], pandas_df['suma_salarios_equipo'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.title('Suma de Valor y Salarios por Equipo (Top 25 Jugadores)')
plt.show()